In [ ]:
import requests

def get_tdc_banxico():
    # !Obtiene los tipos de cambio del Banco de México para EUR/USD, EUR/MXN, GBP/USD y GBP/MXN.
    # Token generado desde Banxico
    token = "36662b701117b841f269b53fda9a936a029bafee8aa375a149b41a67617ccba5"

    # SF57922 = 	Tipos de Cambio para Revalorización de Balance del Banco de México, EUR U.Mon.Europea (EUR/Euro 4/), Dólares por divisa
    # SF57923 = 	Tipos de Cambio para Revalorización de Balance del Banco de México, EUR U.Mon.Europea (EUR/Euro 4/), Tipo en Pesos
    # SF57814 = 	GBP Gran Bretaña (Libra esterlina), Dólares por Divisa
    # SF57815 = 	GBP Gran Bretaña (Libra esterlina), Tipo en Pesos

    # ID de series a consultar
    series_ids = ["SF57922", "SF57923", "SF57814", "SF57815"]
    series_str = ",".join(series_ids)

    # Endpoint REST
    url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{series_str}/datos/oportuno"

    # Encabezados con token de autenticación
    headers = {
        "Bmx-Token": token
    }

    # Hacer la solicitud
    response = requests.get(url, headers=headers)

    # Validar respuesta
    if response.status_code == 200:
        data = response.json()
        series = data.get("bmx", {}).get("series", [])
        tc = ""  # Initialize tc variable
        for serie in series:
            # Si serie_id es SF57922 concatenar con "EUR/USD" pero si es SF57923 concatenar con "EUR/MXN" 
            # pero si es SF57814 concatenar con "GBP/USD" y si es SF57815 concatenar con "GBP/MXN"
            if serie["idSerie"] == "SF57922":
                serie["idSerie"] += " EUR/USD"
            elif serie["idSerie"] == "SF57923":
                serie["idSerie"] += " EUR/MXN"
            elif serie["idSerie"] == "SF57814":
                serie["idSerie"] += " GBP/USD"
            elif serie["idSerie"] == "SF57815":
                serie["idSerie"] += " GBP/MXN"
            # Extraer información de la serie
            serie_id = serie["idSerie"]
            nombre = serie["titulo"]
            valor = serie["datos"][0]["dato"]
            fecha = serie["datos"][0]["fecha"]
            #Concatenear serie_id, nombre, valor y fecha para que se una sola variable
            if tc:  # If tc already has content, add separator
                tc = f"{serie_id}:{valor}:{fecha}" + ";" + tc
            else:  # First iteration
                tc = f"{serie_id}:{valor}:{fecha}"
        return tc
    else:
        return f"Error al consultar Banxico: {response.status_code}"


In [ ]:
tc = get_tdc_banxico()
# Print the result
print(tc)

In [ ]:
import requests
from datetime import datetime, timedelta


# !Obtiene los tipos de cambio del Banco de México para "Para solventar obligaciones" y 
# !"Determinación" para el día actual y el día anterior.
# Token de Banxico
token = "5f05e502aba65738d90d9ee9c1ccb65ab52f100d37499a42404c7f9bfdb1dc64"

# Fecha actual y fecha anterior en formato YYYY-MM-DD
fecha_actual = datetime.now().strftime("%Y-%m-%d")


fecha_ayer = (datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d")

# Encabezados
headers = {
    "Bmx-Token": token
}

# --- Consulta 1: SF60653 y SF43718 para el día actual ---
series_dia_actual = ["SF60653", "SF43718"]
series_str_actual = ",".join(series_dia_actual)
url_actual = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{series_str_actual}/datos/{fecha_actual}/{fecha_actual}"

response_actual = requests.get(url_actual, headers=headers)

if response_actual.status_code == 200:
    data = response_actual.json()
    series = data.get("bmx", {}).get("series", [])
    tc = ""
    for serie in series:
        if serie["idSerie"] == "SF60653":
            serie["titulo"] = "Para solventar obligaciones"
        elif serie["idSerie"] == "SF43718":
                serie["titulo"] = "Determinacion"
        nombre = serie["titulo"]
        try:
            # Si hay datos, extraer el valor y la fecha
            valor = serie["datos"][0]["dato"]
            fecha = serie["datos"][0]["fecha"]
        except:
            # Si no hay datos, asignar valores por defecto
            valor = "N/E"
            fecha = datetime.now().strftime("%d/%m/%Y")
        #Concatenear serie_id, nombre, valor y fecha para que se una sola variable
        if tc:  # If tc already has content, add separator
            tc = f"{nombre}:{valor}:{fecha}" + ";" + tc
        else:  # First iteration
            tc = f"{nombre}:{valor}:{fecha}"

else:
    t = f"Error en consulta actual: {response_actual.status_code}"

    # --- Consulta 2: Sólo SF43718 para el día anterior ---
serie_ayer = "SF43718"
url_ayer = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{serie_ayer}/datos/{fecha_ayer}/{fecha_ayer}"

response_ayer = requests.get(url_ayer, headers=headers)

if response_ayer.status_code == 200:
    data = response_ayer.json()
    tmp = "0"
    serie = data.get("bmx", {}).get("series", [])[0]
    serie["titulo"] = "DOF"
    nombre = serie["titulo"]
    try:
        valor = serie["datos"][0]["dato"]
    except:
        valor = "N/E"
    tdc = f"{nombre}:{valor}:{tmp}"
    tmp =  f"{tc};{tdc}"

    print(tmp)
    
    # Escribir la variable tmp en un archivo de texto
    try:
        with open("tipos_de_cambio.txt", "w", encoding="utf-8") as archivo:
            archivo.write(tmp)
        print("Archivo 'tipos_de_cambio.txt' creado exitosamente.")
    except Exception as e:
        print(f"Error al escribir el archivo: {e}")
        
else:
    tmp = f"Error en consulta de ayer: {response_ayer.status_code}"
    print(tmp)
    
    # Escribir el mensaje de error en el archivo también
    try:
        with open("tipos_de_cambio.txt", "w", encoding="utf-8") as archivo:
            archivo.write(tmp)
        print("Archivo 'tipos_de_cambio.txt' creado con mensaje de error.")
    except Exception as e:
        print(f"Error al escribir el archivo: {e}")

In [3]:
import requests
import xlwings as xw
from datetime import datetime, timedelta, date

# === Parámetros ===
archivo_excel = r"C:\RPA_Process\Timbrado\Refacturacion_regular_v2.xlsm"
nombre_hoja = "TC"
token = "5f05e502aba65738d90d9ee9c1ccb65ab52f100d37499a42404c7f9bfdb1dc64"

# === Abrir Excel y hoja ===
app = xw.App(visible=False)
wb = app.books.open(archivo_excel)
ws = wb.sheets[nombre_hoja]

# === Obtener la última fecha en columna A ===
col_a = ws.range("A2:A100000").value
fechas = [f for f in col_a if isinstance(f, (datetime, date))]

ultima_fecha = max(fechas).date() if fechas else (datetime.today().date() - timedelta(days=2))
fecha_inicio = ultima_fecha + timedelta(days=1)
fecha_fin = datetime.today().date()

if fecha_inicio > fecha_fin:
    print("No hay fechas nuevas para consultar.")
    wb.close()
    app.quit()
    exit()

# === Consulta SF60653 (Solventar) y SF43718 (Determinación) ===
headers = {"Bmx-Token": token}
series_ids = ["SF60653", "SF43718"]
series_str = ",".join(series_ids)
url = f"https://www.banxico.org.mx/SieAPIRest/service/v1/series/{series_str}/datos/{fecha_inicio - timedelta(days=1)}/{fecha_fin}"
response = requests.get(url, headers=headers)

# === Procesar series y generar DOF basado en SF43718 del día anterior ===
datos_por_serie = {
    "SF60653": {},  # Para solventar obligaciones
    "SF43718": {},  # Determinación
    "DOF": {}       # DOF (SF43718 de día anterior)
}

if response.status_code == 200:
    series = response.json().get("bmx", {}).get("series", [])
    for serie in series:
        id_serie = serie["idSerie"]
        for d in serie["datos"]:
            fecha = datetime.strptime(d["fecha"], "%d/%m/%Y").date()
            valor = d["dato"]
            datos_por_serie[id_serie][fecha] = valor

            # Si es determinación, usarla como DOF para día siguiente
            if id_serie == "SF43718":
                fecha_dof = fecha + timedelta(days=1)
                if fecha_dof not in datos_por_serie["DOF"]:
                    datos_por_serie["DOF"][fecha_dof] = valor
else:
    print(f"❌ Error en consulta Banxico: {response.status_code}")
    wb.close()
    app.quit()
    exit()

# === Rango de fechas a insertar ===
registros = {}
dia = fecha_inicio
while dia <= fecha_fin:
    registros[dia] = {
        "Determinacion": datos_por_serie["SF43718"].get(dia, "N/E"),
        "Solventar": datos_por_serie["SF60653"].get(dia, "N/E"),
        "DOF": datos_por_serie["DOF"].get(dia, "N/E")
    }
    dia += timedelta(days=1)

# === Insertar en Excel debajo del último registro ===
ultima_fila = ws.range("A" + str(ws.cells.last_cell.row)).end("up").row + 1

for fecha, valores in sorted(registros.items()):
    fila = [
        fecha.strftime("%d/%m/%Y"),
        valores["Determinacion"],
        valores["DOF"],
        valores["Solventar"]
    ]
    ws.range(f"A{ultima_fila}").value = fila
    ultima_fila += 1

# === Guardar y cerrar Excel ===
wb.save()
wb.close()
app.quit()

print(f"✅ Tipos de cambio agregados de {fecha_inicio} a {fecha_fin} en '{archivo_excel}'.")


✅ Tipos de cambio agregados de 2025-04-30 a 2025-07-21 en 'C:\RPA_Process\Timbrado\Refacturacion_regular_v2.xlsm'.


In [5]:
import requests
from bs4 import BeautifulSoup

# URL del portal de tipos de cambio
url = "https://www.banxico.org.mx/tipcamb/tipCamMIAction.do?idioma=sp"

# Realiza la petición HTTP
response = requests.get(url)

# Verifica si fue exitosa
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")

    # Encuentra la tabla con los datos de tipo de cambio
    table = soup.find("table", {"class": "renglonNon"})

    if table:
        rows = table.find_all("tr")
        print("Fecha\t\tDólar FIX\tDólar Interbancario (venta)\tEuro")
        for row in rows[1:]:  # omitir encabezado
            cols = [td.get_text(strip=True) for td in row.find_all("td")]
            if len(cols) >= 4:
                fecha = cols[0]
                dolar_fix = cols[1]
                interbancario = cols[2]
                euro = cols[3]
                print(f"{fecha}\t{dolar_fix}\t\t{interbancario}\t\t\t{euro}")
    else:
        print("⚠️ No se encontró la tabla de datos.")
else:
    print(f"❌ Error al acceder al portal: {response.status_code}")


⚠️ No se encontró la tabla de datos.
